# Welcome to the Santa 2023 - The Polytope Permutation Puzzle! #

Santa's tree is adorned with many-colored, puzzle-like decorations of all shapes and sizes. Unfortunately, the colors on these decorations have gotten all mixed up! In this competition, you're challenged to set the decorations right in the fewest number of moves.

This notebook will give you a quick introduction to the competition.

# Setup #

In [2]:
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation


data_dir = Path("")

puzzle_info = pd.read_csv(data_dir / 'puzzle_info.csv', index_col='puzzle_type')
# Parse allowed_moves
puzzle_info['allowed_moves'] = puzzle_info['allowed_moves'].apply(literal_eval)

puzzles = pd.read_csv(data_dir / 'puzzles.csv', index_col='id')
# Parse color states
puzzles = puzzles.assign(
    initial_state=lambda df: df['initial_state'].str.split(';'),
    solution_state=lambda df: df['solution_state'].str.split(';')
)

sample_submission = pd.read_csv(data_dir / 'sample_submission.csv', index_col='id')

# Globe Puzzles #
A `globe` puzzle is a sphere with cuts along lines of latitude and longitude. If you poke a hole at the North and South poles, cut along the meridian, and spread it out, it will look like a grid:

```
  | f0  f1  f2  f3  f4  f5  f6  f7
--+-------------------------------
r0| 0   1   2   3   4   5   6   7
r1| 8   9   10  11  12  13  14  15
r2| 16  17  18  19  20  21  22  23
r3| 24  25  26  27  28  29  30  31
```

More precisely, a `globe_M/N` is a sphere with `M` lateral cuts (through latitude) and N radial cuts (through longitude). This gives a `(M + 1) x (2 N)` grid of positions. You could format a color state into a grid like this with something like `np.asarray(state).reshape(m+1, 2*n)`. The above is a `globe_3/4` puzzle.

In [3]:
# Get a solution_state
ss_globe34 = puzzles.query("puzzle_type == 'globe_1/8'").iloc[0]['initial_state']

# Reshape into a grid
np.asarray(ss_globe34).reshape(1+1, 2*8)

array([['I', 'P', 'O', 'A', 'A', 'D', 'F', 'L', 'J', 'M', 'G', 'M', 'P',
        'F', 'E', 'J'],
       ['E', 'B', 'O', 'G', 'H', 'D', 'N', 'L', 'N', 'I', 'B', 'K', 'C',
        'C', 'K', 'H']], dtype='<U1')

In [4]:
puzzles.query("puzzle_type == 'globe_3/4'")

,puzzle_type,solution_state,initial_state,num_wildcards
id,,,,
358,globe_3/4,"[A, A, C, C, E, E, G, G, A, A, C, C, E, E, G, ...","[C, F, B, F, H, H, E, D, H, E, G, A, G, C, E, ...",2
359,globe_3/4,"[A, A, C, C, E, E, G, G, A, A, C, C, E, E, G, ...","[D, B, G, B, F, E, D, C, F, B, F, E, B, G, H, ...",4
360,globe_3/4,"[A, A, C, C, E, E, G, G, A, A, C, C, E, E, G, ...","[E, E, F, A, D, G, F, H, C, F, A, C, G, B, H, ...",2
361,globe_3/4,"[A, A, C, C, E, E, G, G, A, A, C, C, E, E, G, ...","[C, E, G, H, E, F, F, H, G, E, F, E, B, H, A, ...",0
362,globe_3/4,"[A, A, C, C, E, E, G, G, A, A, C, C, E, E, G, ...","[F, H, D, C, D, E, E, A, A, E, E, F, C, H, D, ...",0
363,globe_3/4,"[A, A, C, C, E, E, G, G, A, A, C, C, E, E, G, ...","[F, D, B, G, H, C, A, C, C, A, H, E, C, F, E, ...",0
364,globe_3/4,"[A, A, C, C, E, E, G, G, A, A, C, C, E, E, G, ...","[E, G, G, H, C, B, B, F, G, H, E, A, C, F, B, ...",0
365,globe_3/4,"[A, A, C, C, E, E, G, G, A, A, C, C, E, E, G, ...","[A, G, B, E, F, G, H, A, B, E, F, C, G, D, A, ...",0
366,globe_3/4,"[A, A, C, C, E, E, G, G, A, A, C, C, E, E, G, ...","[B, C, F, G, B, D, E, H, E, C, F, H, G, B, H, ...",0


In [5]:
# Hemisphere operation (180 flip), latitude rotation

# The top/bottom, top-1/bottom+1.... independent
# all the elements in start = end for palindromic rows
# the number of "displaced elements" on top = on bottom
# in 3 moves, we can do a cyclic rotation of half of the perm for top/bot
# all we need to do is ensure the bottom row (N24..N31 end up exactly where they are in the solution state) -> all elements of the top rwo are forced (except the case where answer is just a rotation of top / bottom, all hemisphere operations cancel)
# the order of elements on the top will always be in reverse, the order on the bottom is always opposite

# For globe 1/8, we can reduce to only three moves -> up shift, down shift, hemisphere (at index 0) shift

In [35]:
all_moves = {}
print("globe_1/8")
for m, p in puzzle_info.loc['globe_1/8', 'allowed_moves'].items():
    all_moves[m] = Permutation(p)
    all_moves['-' + m] = Permutation(p) ** -1

# gstate = all_moves['-r0'](gstate)
gstate = np.array([i for i in range(32)])
print(np.asarray(gstate).reshape(1+1, 2*8))

globe_1/8
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]


In [36]:
# # Get a initial_state
# # gstate = puzzles.query("puzzle_type == 'globe_3/4'").iloc[10]['solution_state']
# # gstate = [i for i in range(16)] + ['x' + str(i) for i in range(16)]

# gstate = np.array([ 9,  7, 17, 11, 30, 14,  5,  0, 16, 31,  6,  4,  1,  8, 12, 13,
#        15, 22, 26, 27,  3, 21, 19, 25, 29, 23, 20, 28,  2, 18, 24, 10])
# np.set_printoptions(linewidth=1000)
# # Reshape into a grid
# print(np.asarray(gstate).reshape(1+1, 2*8))

In [54]:
gstate = all_moves['f1'](gstate)
print(np.asarray(gstate).reshape(1+1, 2*8))

[[ 7  8  9 10 11 12 13 14 15  0  1  2  3  4  5  6]
 [23 24 25 26 27 28 29 30 31 16 17 18 19 20 21 22]]


In [8]:
gstate = all_moves['f7'](gstate)
print(np.asarray(gstate).reshape(1+1, 2*8))

[[ 9 29 25 19 21  3 27 24 18  2 28 20 23  7 17 13]
 [15 16  0  5 14 30 11 12  8  1  4  6 31 22 26 10]]


In [53]:
gstate = all_moves['f0'](gstate)
print(np.asarray(gstate).reshape(1+1, 2*8))

[[ 7 31 30 29 28 27 26 25 24  0  1  2  3  4  5  6]
 [23 15 14 13 12 11 10  9  8 16 17 18 19 20 21 22]]


In [22]:
gstate = all_moves['r0'](gstate)
print(np.asarray(gstate).reshape(1+1, 2*8))

[[22 21 20 19 18 17 16  8  9 10 11 12 13 14 15 23]
 [ 7  6  5  4  3  2  1  0 24 25 26 27 28 29 30 31]]


In [51]:
gstate = all_moves['-r0'](gstate)

print(np.asarray(gstate).reshape(1+1, 2*8))

[[ 9 10 11 12 13 14 15 23 24  0  1  2  3  4  5  6]
 [26 27 28 29 30 31  7  8 16 17 18 19 20 21 22 25]]


In [52]:
gstate = all_moves['-r1'](gstate)

print(np.asarray(gstate).reshape(1+1, 2*8))

[[ 9 10 11 12 13 14 15 23 24  0  1  2  3  4  5  6]
 [25 26 27 28 29 30 31  7  8 16 17 18 19 20 21 22]]


In [24]:
gstate = all_moves['r1'](gstate)

print(np.asarray(gstate).reshape(1+1, 2*8))

[[22 21 20 19 18 17 16  8  9 10 11 12 13 14 15 23]
 [ 5  4  3  2  1  0 24 25 26 27 28 29 30 31  7  6]]


In [14]:
gstate = half_turn(gstate)
print(np.asarray(gstate).reshape(1+1, 2*8))

NameError: name 'half_turn' is not defined

In [158]:
def half_turn(gstate):
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    
    gstate = all_moves['f0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['f0'](gstate)
    
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    return gstate

print(np.asarray(half_turn(gstate)).reshape(1+1, 2*8))

def rev_half_turn(gstate):
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['-r0'](gstate)
    
    gstate = all_moves['f0'](gstate)
    gstate = all_moves['-r0'](gstate)
    gstate = all_moves['f0'](gstate)
    
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    return gstate


print(np.asarray(rev_half_turn(gstate)).reshape(1+1, 2*8))

[[ 1  2  3  4  5  6  7 23  8  9 10 11 12 13 14 15]
 [ 0 16 17 18 19 20 21 22 24 25 26 27 28 29 30 31]]
[[16  0  1  2  3  4  5  6  8  9 10 11 12 13 14 15]
 [17 18 19 20 21 22 23  7 24 25 26 27 28 29 30 31]]


In [159]:
gstate = np.array([i for i in range(32)])

def swap_helper(gstate):
    gstate = half_turn(gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)

    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)

    gstate = half_turn(gstate)
    
    return gstate

swap_helper(gstate)

def swap_starts(gstate):
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['-r1'](gstate)

    gstate = swap_helper(gstate)
    
    gstate = all_moves['-r1'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)
    gstate = all_moves['r0'](gstate)

    return gstate

print(np.asarray(swap_starts(gstate)).reshape(1+1, 2*8))

[[ 1  0  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [17 16 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]


In [160]:
def is_sorted_top_bottom(gstate):
    for i in range(16):
        if gstate[i] >= 16:
            return False
    for i in range(16, 32):
        if gstate[i] < 16:
            return False

    return True

def sort_top_bottom(gstate):
    while not is_sorted_top_bottom(gstate):
        bad_top, bad_bottom = -1, -1

        for i in range(16):
            if gstate[i] >= 16:
                bad_top = i
                break


        for i in range(16, 32):
            if gstate[i] < 16:
                bad_bottom = i - 16
                break

        
        print('A', np.asarray(gstate).reshape(1+1, 2*8))
        # Set up for half turn
        for i in range(bad_top):
            # cyclic shift left
            gstate = all_moves['r0'](gstate)

        for i in range(bad_bottom):
            gstate = all_moves['r1'](gstate)

        for i in range(7):
            gstate = all_moves['-r1'](gstate)


        print('B', np.asarray(gstate).reshape(1+1, 2*8))

        gstate = half_turn(gstate)

        print('C', np.asarray(gstate).reshape(1+1, 2*8))
    return gstate

gstate = sort_top_bottom(gstate)
np.asarray(gstate).reshape(1+1, 2*8)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]])

In [161]:
np.asarray(gstate).reshape(1+1, 2*8)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]])

In [162]:
gstate = adj_rot_bot_idx(gstate, 1)

NameError: name 'adj_rot_bot_idx' is not defined

In [180]:
# def adj_rot_top():
    

a = [i for i in range(32)]

def adj_rot_bot(a):
    a = all_moves['f0'](a)
    a = all_moves['-r0'](a)
    a = all_moves['f0'](a)

    a = all_moves['r1'](a)

    a = all_moves['f0'](a)
    a = all_moves['r0'](a)
    a = all_moves['f0'](a)

    a = all_moves['-r1'](a)
    a = sort_top_bottom(a)

    # for i in range(8):
    #     a = all_moves['r1'](a)
    #     a = all_moves['r0'](a)

    # a = rev_half_turn(a)

    # for i in range(8):
    #     a = all_moves['-r1'](a)
    #     a = all_moves['-r0'](a)
    
    return a



# swaps i, i+1
def adj_rot_bot_idx(a, i):
    for j in range(i):
        a = all_moves['r1'](a)

    a = adj_rot_bot(a)

    for j in range(i):
        a = all_moves['-r1'](a)

    return a
        
# [[ 0  1  2  3  4  5  6  7 14 15  8  9 10 11 12 13]
#  [16 17 18 19 20 21 22 23 26 27 28 29 30 31 24 25]]
def move_two_to_end(a):
    a = adj_rot_bot(a)
    a = adj_rot_bot(a)
    return a

a = adj_rot_bot(a)

A [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 24]
 [17 16 18 19 20 21 22 23 15 25 26 27 28 29 30 31]]
B [[24  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [16 18 19 20 21 22 23 15 25 26 27 28 29 30 31 17]]
C [[ 0  1  2  3  4  5  6 15  7  8  9 10 11 12 13 14]
 [24 16 18 19 20 21 22 23 25 26 27 28 29 30 31 17]]


In [181]:
print(np.asarray(a).reshape(1+1, 2*8))

[[ 0  1  2  3  4  5  6 15  7  8  9 10 11 12 13 14]
 [24 16 18 19 20 21 22 23 25 26 27 28 29 30 31 17]]


In [19]:
# Get a solution_state
ss_globe34 = puzzles.query("puzzle_type == 'globe_3/4'").iloc[10]['solution_state']

# Reshape into a grid
np.asarray(ss_globe34).reshape(3+1, 2*4)

array([['N0', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7'],
       ['N8', 'N9', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15'],
       ['N16', 'N17', 'N18', 'N19', 'N20', 'N21', 'N22', 'N23'],
       ['N24', 'N25', 'N26', 'N27', 'N28', 'N29', 'N30', 'N31']],
      dtype='<U3')

There is a move for each lateral layer (the `r`s) and a move for each half (the `f`s). The `r` moves shift a layer by one position laterally, while the `f` moves shift one half of the globe by a half-twist.

In [20]:
all_moves = {}
print("globe_3/4")
for m, p in puzzle_info.loc['globe_3/4', 'allowed_moves'].items():
    all_moves[m] = Permutation(p)
    print(f"{m}: {Permutation(p)}")

globe_3/4
r0: (31)(0 1 2 3 4 5 6 7)
r1: (31)(8 9 10 11 12 13 14 15)
r2: (31)(16 17 18 19 20 21 22 23)
r3: (24 25 26 27 28 29 30 31)
f0: (31)(0 27)(1 26)(2 25)(3 24)(8 19)(9 18)(10 17)(11 16)
f1: (31)(1 28)(2 27)(3 26)(4 25)(9 20)(10 19)(11 18)(12 17)
f2: (31)(2 29)(3 28)(4 27)(5 26)(10 21)(11 20)(12 19)(13 18)
f3: (31)(3 30)(4 29)(5 28)(6 27)(11 22)(12 21)(13 20)(14 19)
f4: (4 31)(5 30)(6 29)(7 28)(12 23)(13 22)(14 21)(15 20)
f5: (0 29)(5 24)(6 31)(7 30)(8 21)(13 16)(14 23)(15 22)
f6: (0 31)(1 30)(6 25)(7 24)(8 23)(9 22)(14 17)(15 16)
f7: (0 25)(1 24)(2 31)(7 26)(8 17)(9 16)(10 23)(15 18)


In [21]:
# Get a initial_state
initial_state = puzzles.query("puzzle_type == 'globe_1/8'").iloc[0]['initial_state']

# Reshape into a grid
np.asarray(initial_state).reshape(1+1, 2*8)

array([['I', 'P', 'O', 'A', 'A', 'D', 'F', 'L', 'J', 'M', 'G', 'M', 'P',
        'F', 'E', 'J'],
       ['E', 'B', 'O', 'G', 'H', 'D', 'N', 'L', 'N', 'I', 'B', 'K', 'C',
        'C', 'K', 'H']], dtype='<U1')

In [22]:
all_moves = {}
for m, p in puzzle_info.loc['globe_1/8', 'allowed_moves'].items():
    all_moves[m] = np.array(p)
    # all_moves[m] = Permutation(p)
#     print(f"{m}: {Permutation(p)}")
    print(f"{m}: {p}")

r0: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
r1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 16]
f0: [23, 22, 21, 20, 19, 18, 17, 16, 8, 9, 10, 11, 12, 13, 14, 15, 7, 6, 5, 4, 3, 2, 1, 0, 24, 25, 26, 27, 28, 29, 30, 31]
f1: [0, 24, 23, 22, 21, 20, 19, 18, 17, 9, 10, 11, 12, 13, 14, 15, 16, 8, 7, 6, 5, 4, 3, 2, 1, 25, 26, 27, 28, 29, 30, 31]
f2: [0, 1, 25, 24, 23, 22, 21, 20, 19, 18, 10, 11, 12, 13, 14, 15, 16, 17, 9, 8, 7, 6, 5, 4, 3, 2, 26, 27, 28, 29, 30, 31]
f3: [0, 1, 2, 26, 25, 24, 23, 22, 21, 20, 19, 11, 12, 13, 14, 15, 16, 17, 18, 10, 9, 8, 7, 6, 5, 4, 3, 27, 28, 29, 30, 31]
f4: [0, 1, 2, 3, 27, 26, 25, 24, 23, 22, 21, 20, 12, 13, 14, 15, 16, 17, 18, 19, 11, 10, 9, 8, 7, 6, 5, 4, 28, 29, 30, 31]
f5: [0, 1, 2, 3, 4, 28, 27, 26, 25, 24, 23, 22, 21, 13, 14, 15, 16, 17, 18, 19, 20, 12, 11, 10, 9, 8, 7, 6, 5, 29, 30, 31]
f6: [0, 1, 2, 3,

In [170]:
def expand(path, n = 2, m = 16):    
    rows = [[] for _ in range(n)]
    
    def expand_f(x):
        for i in range(n):
            rows[i].append(x)
            rows[i].append('f0')
            rows[i].append(m - x)
    
    for i in path:
        if 'f' in i:
            if '-' in i:
                expand_f(int(i[2:]))
            else:
                expand_f(int(i[1:]))
        else:
            for j in range(n):
                temp = 'r' + str(j)

                if temp == i:
                    rows[j].append(1)
                    
                temp = '-r' + str(j)

                if temp == i:
                    rows[j].append(m-1)

    Rrows = [[] for _ in range(n)]

    for r in range(n):
        run = 0
        for i in rows[r]:
            if type(i) == type(0):
                run += i
            else:
                Rrows[r].append(run)
                Rrows[r].append('f0')
                run = 0
            
        Rrows[r].append(run)
    
    return Rrows

def shrink(rows, n = 2, m = 16):
    for i in range(len(rows[0])):
        mn = int(1e9)
        for j in range(n):
            if type(rows[j][i]) == type(1):
                rows[j][i] %= m
            else:
                mn = min(mn, rows[j][i-1])
        
        for j in range(n):  
            if type(rows[j][i]) == type('string'):
                rows[j][i-1] -= mn
                rows[j][i] = 'f' + str(mn)
                rows[j][i+1] += mn
    
    final_moves = []
    for i in range(len(rows[0])):
        if type(rows[0][i]) == type('string'):
            final_moves.append(rows[0][i])
            continue

        for j in range(n):
            if rows[j][i] > m/2:
                final_moves = final_moves + ['-r' + str(j)] * (m - rows[j][i])
            else:
                final_moves = final_moves + ['r' + str(j)] * (rows[j][i])

    final_final_moves = []
    i = 0
    while i < len(final_moves):
        if i != len(final_moves) - 1 and ('f' in final_moves[i]) and final_moves[i] == final_moves[i+1]:
            i += 2
        else:
            final_final_moves.append(final_moves[i])
            i += 1
    
    return final_final_moves

def ex_red(moves):
    ex = expand(moves)
    return shrink(ex)

# ex = expand(sample_submission['moves'][idx].split('.'), N, M)
input = ['f0', '-r0', 'f0', 'r1', 'f0', 'r0', 'f0', '-r1'] + ['r1'] * 8 + ['r0'] * 8 + ['-r0'] * 8 + ['f0', '-r0', 'f0'] + ['r0'] * 8 + ['-r1'] * 8 + ['-r0'] * 8

input = input + input
ex = expand(input, 2, 16)
print(ex)
res = shrink(ex, 2, 16)
res

[[0, 'f0', 31, 'f0', 16, 'f0', 17, 'f0', 144, 'f0', 31, 'f0', 144, 'f0', 31, 'f0', 16, 'f0', 17, 'f0', 144, 'f0', 31, 'f0', 144], [0, 'f0', 16, 'f0', 17, 'f0', 16, 'f0', 39, 'f0', 16, 'f0', 136, 'f0', 16, 'f0', 17, 'f0', 16, 'f0', 39, 'f0', 16, 'f0', 136]]


['f0',
 '-r0',
 'f0',
 'r1',
 'f0',
 'r0',
 'f0',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'f0',
 '-r0',
 'f0',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'f0',
 '-r0',
 'f0',
 'r1',
 'f0',
 'r0',
 'f0',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'f0',
 '-r0',
 'f0',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1',
 'r1']

In [172]:
cnt_f0s = 0
for i in res:
    if i == 'f0':
        cnt_f0s+=1

print(cnt_f0s)

12


In [173]:
test_globe = np.array([i for i in range(2 * 16)])
for i in input:
    test_globe = all_moves[i](test_globe)

print(np.asarray(test_globe).reshape(1+1, 2*8))

[[ 0  1  2  3  4  5  6  7 14 15  8  9 10 11 12 13]
 [16 17 18 19 20 21 22 23 26 27 28 29 30 31 24 25]]


In [23]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd

In [24]:
@jit(nopython=True, parallel = True, fastmath = True)
def hash_perm(perm):
    base = 9973
    modb = 1000000007
    modc = 1000000009

    B, C = 0, 0
    for i in perm:
        B = (B * base) % modb + i
        C = (C * base) % modc + i

    return str(B) + str(C)

In [25]:
# 1e6 ~ 2 seconds
mx_mem = int(1e7)
mem_idx = 0

last_state = np.zeros(mx_mem, dtype=int)
last_move = np.zeros(mx_mem, dtype=int)
cur_state = np.array([np.array([]) for i in range(mx_mem)])

print(last_state)

[0 0 0 ... 0 0 0]


In [26]:
# M = np.array([all_moves['r0'], all_moves['r1'], all_moves['f0']])
M = np.array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  0,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 16],
       [23, 22, 21, 20, 19, 18, 17, 16,  8,  9, 10, 11, 12, 13, 14, 15,
         7,  6,  5,  4,  3,  2,  1,  0, 24, 25, 26, 27, 28, 29, 30, 31]])

In [27]:
%%time
# @jit(nopython=True, fastmath = True)
def BFS(last_state, last_move, cur_state, M):
    mem_idx = 1
    initial_state = np.arange(32)
    visited = dict()
    visited[hash_perm(initial_state)] = 0

    i = 0
    while mem_idx + 10 < mx_mem:
        state = cur_state[i]
        
        for j in range(3):
            m = M[j]
            next_state = state[m]
            next_hash = hash_perm(next_state)
            
            if(next_hash in visited):
                continue
                
            visited[next_hash] = mem_idx
            last_state[mem_idx] = i
            last_move[mem_idx] = j
            
            mem_idx += 1
        i += 1
        
    return visited

a = BFS(last_state, last_move, cur_state, M)

/opt/homebrew/lib/python3.11/site-packages/numba/core/typed_passes.py:334: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../var/folders/kd/99328vjn0zlc1kls78_rwvdh0000gn/T/ipykernel_28049/3020915523.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


IndexError: index 1 is out of bounds for axis 0 with size 0

In [28]:
import cProfile
cProfile.run('BFS()')

         3 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




TypeError: BFS() missing 4 required positional arguments: 'last_state', 'last_move', 'cur_state', and 'M'

In [ ]:
def expandPath(

# Submissions #

The submission format requires moves in a sequence to be delimited by a period `.`. Indicate the inverse of a named move with a preceeding `-`. Moves are applied from left to right.

The `sample_submission.csv` contains a baseline solution for each puzzle.

In [12]:
sample_submission

,moves
id,
0,r1.-f1
1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...
393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....


Let's check that the given sequence for the first puzzle is actually a solution.

In [13]:
def apply_sequence(sequence, moves, state):
    """Apply a sequence of moves in array form to a color state."""
    state = np.asarray(state)
    for m in sequence.split('.'):
        state = state[moves[m]]
    return state


# Convert allowed_moves to dict and add inverse moves
all_moves = puzzle_info.loc[:, 'allowed_moves'].to_dict()
for ptype, moves in all_moves.copy().items():
    for m, arr in moves.copy().items():
        all_moves[ptype][f"-{m}"] = np.argsort(arr).tolist()


# Get info for the first puzzle
solution_state = puzzles.iloc[0, 1]
initial_state = puzzles.iloc[0, 2]
baseline_solution = sample_submission.loc[0, 'moves']

state = apply_sequence(baseline_solution, all_moves['cube_2/2/2'], initial_state)
np.array_equal(state, solution_state)

True

In this case, the resulting state is exactly equal to the required solution state. For puzzles with wildcards, there can be an allowed number of differences. See the competitions [Evaluation](https://www.kaggle.com/competitions/santa-2023/overview/evaluation) page for more info.

# Good luck! #

We hope this introduction was informative! Here are some places to follow up for more information:
- [The Complexity Dynamics of Magic Cubes and Twisty Puzzles](https://resplendence.org/dhushara.com/cubes/cubes.pdf)
- [Twisty Puzzle Museum](https://twistypuzzles.com/app/museum/museum_search.php)
- [Analyzing Rubik's Cube with GAP](https://www.gap-system.org/Doc/Examples/rubik.html) - You could try reproducing this with [SAGE](https://www.sagemath.org/), if you prefer.